In [4]:
import numpy as np
import pandas as pd
import cv2
import math

In [8]:
import os

def create_array():
    images = []
    target = []

    directory = "E:/CoMoFoD_small_v2"
    image_list = os.listdir(directory)

    for name in image_list:
        
        location = os.path.join(directory, name)
        image = cv2.imread(location)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
        
        if 'F' in name:
            images.append(image)
            target.append(1)
        elif 'O' in name:
            images.append(image)
            target.append(0)
                
    return images, target

In [9]:
images, target = create_array()

In [5]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import cv2
import pickle
from keras.initializers import glorot_normal
from keras.initializers import Constant
from sklearn.utils import shuffle

In [11]:
train_data, train_target = shuffle(images, target, random_state = 2)

In [12]:
from matplotlib import pyplot as plt
plt.figure()
g = cv2.cvtColor(train_data[20], cv2.COLOR_BGR2RGB)
plt.imshow(g)
plt.show()

<Figure size 640x480 with 1 Axes>

In [13]:
train_target = np.array(train_target)
train_data = np.array(train_data)
train_target = np.expand_dims(train_target, 1)

In [14]:
with open('image_1.pkl', 'wb') as f1:
    pickle.dump(train_data, f1)
with open('image_target_1.pkl', 'wb') as f1:
    pickle.dump(train_target, f1)

In [6]:
with open('image_1.pkl','rb') as f:
    train_data = pickle.load(f)

with open('image_target_1.pkl','rb') as f:
    train_target = pickle.load(f)

In [7]:
train_data[0].shape

(224, 224, 3)

In [8]:
train_generator = ImageDataGenerator(featurewise_center=False,
                                    featurewise_std_normalization=False,
                                    rotation_range=10,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.1,
                                    rescale=1.0/255.0,
                                    horizontal_flip=True)

In [9]:
test_generator = ImageDataGenerator(rescale=1.0/255.0)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target, shuffle = True, random_state = 3)

In [11]:
x_train.shape, y_train.shape

((7500, 224, 224, 3), (7500, 1))

In [12]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [13]:
from keras import applications
from keras.layers import Input

In [14]:
x_train.shape

(7500, 224, 224, 3)

In [15]:
import keras
import tensorflow as tf
import keras.backend as K

def build_filters():
    filters = []
    ksize = 31
    for theta in np.arange(0, np.pi, np.pi / 16):
        kern = cv2.getGaborKernel((ksize, ksize), 4.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        kern /= 1.5*kern.sum()
        filters.append(kern)
    return filters

def image_func(img, filters):
#     imgm=cv2.medianBlur(img, 3)
#     img= np.expand_dims(imgm, axis = 2) - img
#     return img.astype('float32')
    accum = np.zeros_like(img)
    for kern in filters:
        fimg = cv2.filter2D(img, cv2.CV_8UC3, kern)
        np.maximum(accum, fimg, accum)
    return accum

def image_tensor_func(img4d) :
    results = []
    filters = build_filters()
    for img3d in img4d :
        rimg3d = image_func(img3d , filters)
        results.append( np.expand_dims( rimg3d, axis=0 ) )
    return np.concatenate( results, axis = 0 )

class Gabor(keras.layers.Layer ) :
    def call( self, xin )  :
        xout = tf.py_func( image_tensor_func, 
                           [xin],
                           'float32',
                           stateful=False,
                           name='cvOpt')
        xout = K.stop_gradient( xout )
        xout.set_shape( [None, 224, 224, 3]) 
        return xout
    def compute_output_shape( self, sin ) :
        return (None, 224, 224, 3)

In [16]:
from keras.models import Model

image_input = Input(shape = (224,224,3))

my_layer = Gabor(name = 'gabor')(image_input)

vggmodel = applications.VGG16(input_tensor=my_layer, weights='imagenet', include_top=True)


In [17]:
last_layer = vggmodel.get_layer('fc2').output
out = Dense(2, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

# for layer in custom_vgg_model.layers[:-1]:
# 	layer.trainable = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
gabor (Gabor)                (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________

In [22]:
for layer in custom_vgg_model.layers[2:-1]:
	layer.trainable = False

In [23]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

hist = custom_vgg_model.fit(x_train, y_train, batch_size=20, epochs=12, verbose=1, validation_data=(x_test, y_test))

Train on 7500 samples, validate on 2500 samples
Epoch 1/12
7500/7500 [==============================] - 958s 128ms/step - loss: 0.7370 - acc: 0.6513 - val_loss: 0.5289 - val_acc: 0.7488
Epoch 2/12
7500/7500 [==============================] - 977s 130ms/step - loss: 0.4824 - acc: 0.7728 - val_loss: 0.3819 - val_acc: 0.8244
Epoch 3/12
7500/7500 [==============================] - 974s 130ms/step - loss: 0.4040 - acc: 0.8144 - val_loss: 0.3316 - val_acc: 0.8360
Epoch 4/12
7500/7500 [==============================] - 976s 130ms/step - loss: 0.3583 - acc: 0.8496 - val_loss: 0.3371 - val_acc: 0.8580
Epoch 5/12
7500/7500 [==============================] - 967s 129ms/step - loss: 0.3096 - acc: 0.8739 - val_loss: 0.2709 - val_acc: 0.8920
Epoch 6/12
7500/7500 [==============================] - 961s 128ms/step - loss: 0.3488 - acc: 0.8555 - val_loss: 0.2548 - val_acc: 0.8948
Epoch 7/12
7500/7500 [==============================] - 975s 130ms/step - loss: 0.2710 - acc: 0.8932 - val_loss: 0.2618 - va

In [ ]:
k = np.array([[1, 2, 3], [2, 3, 4], [5, 6, 6]])